In [1]:
from openbb_terminal.sdk import openbb
import pandas as pd
import os
from dotenv import load_dotenv


Forecasting Toolkit is disabled. To use the Forecasting features please install the toolkit following the instructions here: https://my.openbb.co/app/sdk/installation

Portfolio Optimization Toolkit is disabled. To use the Optimization features please install the toolkit following the instructions here: https://my.openbb.co/app/sdk/installation

In [2]:
load_dotenv()

sp500 = pd.read_csv("sp500.csv")


In [3]:
openbb.keys.fmp(os.environ["FMP_APIKEY"])


'Defined, test passed'

In [4]:
sp500.columns = [x.lower().replace(" ", "_") for x in sp500.columns]
sp500


,symbol,security,gics_sector,gics_sub-industry,headquarters_location,date_added,cik,founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,1800,1888
3,ABBV,AbbVie,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989
...,...,...,...,...,...,...,...,...
498,YUM,Yum! Brands,Consumer Discretionary,Restaurants,"Louisville, Kentucky",1997-10-06,1041061,1997
499,ZBRA,Zebra Technologies,Information Technology,Electronic Equipment & Instruments,"Lincolnshire, Illinois",2019-12-23,877212,1969
500,ZBH,Zimmer Biomet,Health Care,Health Care Equipment,"Warsaw, Indiana",2001-08-07,1136869,1927
501,ZION,Zions Bancorporation,Financials,Regional Banks,"Salt Lake City, Utah",2001-06-22,109380,1873


In [5]:
sp500["symbol"] = sp500["symbol"].str.strip()


In [6]:
data = pd.DataFrame(sp500["symbol"].iloc[0:5])
data


,symbol
0,MMM
1,AOS
2,ABT
3,ABBV
4,ACN


In [7]:
def get_fa_ratios(symbol):
    fa_ratios = openbb.stocks.fa.ratios(symbol=symbol).iloc[:, 0]
    return fa_ratios[["Payables turnover", "Receivables turnover", "Debt equity ratio", "Cash ratio", "Current ratio", "Debt ratio"]]

data[["apt_rate", "art_rate", "de_ratio", "cash_ratio", "current_ratio", "da_ratio"]] = data["symbol"].apply(get_fa_ratios)
data[["apt_days", "art_rate"]] = 360 / data[["apt_rate", "art_rate"]]


In [8]:
def get_balance_sheet_data(symbol):
    balance_sheet = openbb.stocks.fa.balance(symbol=symbol).iloc[:, 0]
    return balance_sheet[["Long-term debt", "Total assets", "Total current assets"]]

data[["lt_debt", "total_assets", "total_current_assets"]] = data["symbol"].apply(get_balance_sheet_data)
data["lt_debt_to_assets"] = data["lt_debt"] / data["total_assets"]
data["current_asset_ratio"] = data["total_current_assets"] / data["total_assets"]


In [9]:
data


,symbol,apt_rate,art_rate,de_ratio,cash_ratio,current_ratio,da_ratio,apt_days,lt_debt,total_assets,total_current_assets,lt_debt_to_assets,current_asset_ratio
0,MMM,6.042099,47.664846,1.096818,0.383808,1.542371,0.348725,59.581947,1.400100e+10,4.645500e+10,1.468800e+10,0.301388,0.316177
1,AOS,3.873921,55.737233,0.209933,0.418754,1.748769,0.110104,92.929093,3.345000e+08,3.332300e+09,1.633700e+09,0.100381,0.490262
2,ABT,4.154982,73.240327,0.482909,0.638001,1.628511,0.237997,86.642984,1.452200e+10,7.443800e+10,2.522400e+10,0.195089,0.338859
3,ABBV,5.935242,69.787439,3.667034,0.311497,0.963606,0.455827,60.654646,5.913500e+10,1.388050e+11,2.846300e+10,0.426029,0.205057
4,ACN,17.413539,68.658043,0.122565,0.502250,1.298344,0.061450,20.673569,4.309300e+07,5.124530e+10,2.338193e+10,0.000841,0.456275
